# Orders

In [1]:
import requests
import pandas as pd
import json
import time
from datetime import date

In [ ]:
# Склад остатки - stocks?dateFrom=2023-04-12
# Заказы        - orders?dateFrom=2023-02-09&flag=0
# Продажи       - sales?dateFrom=2023-02-09&flag=0
# Поставки      - incomes?dateFrom=2023-03-19
# Отчет о продажах по реализации - reportDetailByPeriod?dateFrom=2023-02-09&dateTo=2023-02-28

In [2]:
# Выгружаем данные
# На выходе получается строка с информацие в виде словаря
url = "https://statistics-api.wildberries.ru/api/v1/supplier/orders?dateFrom=2023-01-01&flag=0"
payload = {}
headers = {
  'Authorization': ''
}
response = requests.request("GET", url, headers=headers, data=payload)
print(response.text)

[{"date":"2023-02-22T13:18:54","lastChangeDate":"2023-02-22T23:41:08","supplierArticle":"Этнический халат","techSize":"XS-L","barcode":"2037488839493","totalPrice":6400,"discountPercent":41,"warehouseName":"Коледино","oblast":"Москва","incomeID":10960235,"odid":9001947819539,"nmId":147263932,"subject":"Халаты домашние","category":"Одежда","brand":"","isCancel":false,"cancel_dt":"0001-01-01T00:00:00","gNumber":"96772691678002781140","sticker":"","srid":"17823514073508166.1.0"},{"date":"2023-02-23T07:29:53","lastChangeDate":"2023-02-23T12:35:43","supplierArticle":"Этнический халат","techSize":"XS-L","barcode":"2037488839493","totalPrice":6400,"discountPercent":41,"warehouseName":"Коледино","oblast":"Москва","incomeID":10960235,"odid":9001954570071,"nmId":147263932,"subject":"Халаты домашние","category":"Одежда","brand":"","isCancel":false,"cancel_dt":"0001-01-01T00:00:00","gNumber":"200213017041704931","sticker":"","srid":"9fe31f11007e4d46b63772885572e81e"},{"date":"2023-02-23T08:03:16",

# Предварительная обработка таблицы

In [3]:
# Данные в переменную
str_total = response.text  # достаём данные в строку
str_total

'[{"date":"2023-02-22T13:18:54","lastChangeDate":"2023-02-22T23:41:08","supplierArticle":"Этнический халат","techSize":"XS-L","barcode":"2037488839493","totalPrice":6400,"discountPercent":41,"warehouseName":"Коледино","oblast":"Москва","incomeID":10960235,"odid":9001947819539,"nmId":147263932,"subject":"Халаты домашние","category":"Одежда","brand":"","isCancel":false,"cancel_dt":"0001-01-01T00:00:00","gNumber":"96772691678002781140","sticker":"","srid":"17823514073508166.1.0"},{"date":"2023-02-23T07:29:53","lastChangeDate":"2023-02-23T12:35:43","supplierArticle":"Этнический халат","techSize":"XS-L","barcode":"2037488839493","totalPrice":6400,"discountPercent":41,"warehouseName":"Коледино","oblast":"Москва","incomeID":10960235,"odid":9001954570071,"nmId":147263932,"subject":"Халаты домашние","category":"Одежда","brand":"","isCancel":false,"cancel_dt":"0001-01-01T00:00:00","gNumber":"200213017041704931","sticker":"","srid":"9fe31f11007e4d46b63772885572e81e"},{"date":"2023-02-23T08:03:16"

In [4]:
# Создаём первую строку датафрэйма
# Убираем лишние символы, разбиваем, берём первый заказ, добавляем символы для словаря
s_first = '{' + str_total.replace('[', '').replace(']', '').split('},{')[0].replace('{', '').replace('}', '') + '}'
# преобразуем словарь
json_acceptable_string = s_first.replace('"', "\"")
data = json.loads(json_acceptable_string)
# словарь в дф
df = pd.DataFrame([data])
df

,date,lastChangeDate,supplierArticle,techSize,barcode,totalPrice,discountPercent,warehouseName,oblast,incomeID,odid,nmId,subject,category,brand,isCancel,cancel_dt,gNumber,sticker,srid
0,2023-02-22T13:18:54,2023-02-22T23:41:08,Этнический халат,XS-L,2037488839493,6400,41,Коледино,Москва,10960235,9001947819539,147263932,Халаты домашние,Одежда,,False,0001-01-01T00:00:00,96772691678002781140,,17823514073508166.1.0


In [5]:
# Проходимся циклом по строке и дополняем датафрэйм
# Запускать 1 раз!
str_need = str_total.split('},{')
for i in str_need[1:]:  # Со второго элемента, так как первый уже закинул в дф
    i = '{' + i.replace('[', '').replace(']', '').replace('{', '').replace('}', '') + '}'
    json_acceptable_string = i.replace('"', "\"")
    data = json.loads(json_acceptable_string)
    df_t = pd.DataFrame([data])
    df = pd.concat([df, df_t])
df = df.reset_index(drop=True)
df.head(3)

,date,lastChangeDate,supplierArticle,techSize,barcode,totalPrice,discountPercent,warehouseName,oblast,incomeID,odid,nmId,subject,category,brand,isCancel,cancel_dt,gNumber,sticker,srid
0,2023-02-22T13:18:54,2023-02-22T23:41:08,Этнический халат,XS-L,2037488839493,6400.0,41,Коледино,Москва,10960235,9001947819539,147263932,Халаты домашние,Одежда,,False,0001-01-01T00:00:00,96772691678002781140,,17823514073508166.1.0
1,2023-02-23T07:29:53,2023-02-23T12:35:43,Этнический халат,XS-L,2037488839493,6400.0,41,Коледино,Москва,10960235,9001954570071,147263932,Халаты домашние,Одежда,,False,0001-01-01T00:00:00,200213017041704931,,9fe31f11007e4d46b63772885572e81e
2,2023-02-23T08:03:16,2023-02-23T12:41:55,Синий халат,XS-L,2037488779201,5800.0,36,Коледино,Москва,10960235,9001954710126,147263461,Халаты домашние,Одежда,,False,0001-01-01T00:00:00,96805883299484393157,,1501293d1455454cb2551f51415fc133


In [19]:
date - дата
gNumber - Номер заказа. Объединяет все позиции одного заказа.
supplierArticle - артикуль
techSize - размер
totalPrice - Цена до согласованной итоговой скидки/промо/спп.
discountPercent - Согласованный итоговый дисконт.
incomeID - Номер поставки (от продавца на склад)
! odid - Уникальный идентификатор позиции заказа. Может использоваться для поиска соответствия между заказами и продажами.
nmId - Артикул WB
subject - Предмет
isCancel - Отмена заказа. true - заказ отменен до оплаты.
cancel_dt - Дата и время отмены заказа
srid - Уникальный идентификатор заказа, функционально аналогичный odid/rid

SyntaxError: invalid syntax (4055317334.py, line 4)

In [6]:
# Создали колонку с датами заказа / отмены заказа
df = df.assign(date_corr = df.date.apply(lambda x: x.split('T')[0]))
df = df.assign(cancel_date = df.cancel_dt.apply(lambda x: x.split('T')[0]))

In [7]:
# Даты в тип с датой покупки
df['date_corr'] = pd.to_datetime(df['date_corr'],format='%Y-%m-%d')
# С колонкой даты по возврату нужно немного помучиться
# Невозможно загнать в дату число '0001-01-01'
def date_canc_func(row):
    if row == '0001-01-01':
        return '2023-02-22'
    else:
        return row
df['cancel_date'] = df['cancel_date'].apply(date_canc_func)
df['cancel_date'] = pd.to_datetime(df['cancel_date'],format='%Y-%m-%d')

In [8]:
# Оставляем нужные колонки
df = df[['date_corr', 'gNumber', 'srid', 'supplierArticle', 'techSize', 'totalPrice', 'discountPercent', 'incomeID',  \
         'odid', 'nmId', 'subject', 'isCancel', 'cancel_date']].copy()

In [9]:
# Добавали цены со скидкой - priceWithDiscount
df['priceWithDiscount'] = df.totalPrice * (1 - df.discountPercent / 100)

In [10]:
# Колонку с отменой откорректировали
df['isCancel'] = df['isCancel'] * 1

In [12]:
# Сохраним файл
today = date.today()
df.to_excel(f'orders_{today}.xlsx', index=False)
df.to_csv(f'orders_{today}.csv', index=False)

# Исследование данных

In [46]:
# Заказы, где несколько позиций
df.groupby('gNumber', as_index=False).agg({'supplierArticle' : 'count'})  \
    .sort_values('supplierArticle', ascending=False).head(10)

,gNumber,supplierArticle
76,9012704706841939191,4
146,96992784473039596159,3
100,92889767614462014074,3
45,5879849442884060468,2
1,1105333839752510442,2
61,735160258708416067,2
165,9890115709619529856,2
33,4167807836533536,2
17,2859805610539408127,2
117,94527030776146887138,1


In [14]:
# Отменённые заказы
cancelled = df[df.isCancel == True]
cancelled

,date_corr,gNumber,srid,supplierArticle,techSize,totalPrice,discountPercent,incomeID,odid,nmId,subject,isCancel,cancel_date,priceWithDiscount
21,2023-03-01,5130338983342920726,2e0e15f9028d4c04ac858e617113e588,Синий халат,XS-L,5800.00,36,10960235,9002020765959,147263461,Халаты домашние,1,2023-03-03,3712.0000
26,2023-03-04,91289940096818655628,b6a448df0b3f4fb2bdd62eb296de13b3,Синий халат,XS-L,4803.13,36,10960235,9002051705309,147263461,Халаты домашние,1,2023-03-07,3074.0032
29,2023-03-08,8849912591178972883,12918656074130752.0.0,Этнический халат,XS-L,6400.00,41,0,9002101113782,147263932,Халаты домашние,1,2023-03-10,3776.0000
31,2023-03-11,93460505466031179453,e6509d74b8224f0da6692a021bee2fa4,Этнический халат,XS-L,6400.00,48,10960235,9002126207609,147263932,Халаты домашние,1,2023-03-12,3328.0000
36,2023-03-10,4027202863189022883,d80b2d99f4ff406fbfa46e688577f8d7,Этнический халат,XS-L,6400.00,41,10960235,9002114019805,147263932,Халаты домашние,1,2023-03-13,3776.0000
37,2023-03-12,8826096817396535016,15628841574293186.2.0,Шелковая пижама белые листья,XS-L,6700.00,62,10960235,9002138456427,147271347,Пижамы,1,2023-03-13,2546.0000
38,2023-03-10,2821973071259241728,55632537074198579.1.0,Шелковая пижама белые листья,XS-L,6700.00,49,10960235,9002114995693,147271347,Пижамы,1,2023-03-13,3417.0000
42,2023-03-12,9946999397790708657,18124248074278511.0.0,Синий халат,XS-L,5800.00,57,10960235,9002133073778,147263461,Халаты домашние,1,2023-03-15,2494.0000
44,2023-03-13,94791398545829743066,8b8c5f525e204413a9277e80840e1355,Этнический халат,XS-L,6400.00,48,10960235,9002148289742,147263932,Халаты домашние,1,2023-03-16,3328.0000
45,2023-03-15,97883735890419337050,5412c0e9722049b4bf2a2de18546970d,Этнический халат,XS-L,6400.00,48,10960235,9002166892667,147263932,Халаты домашние,1,2023-03-17,3328.0000


In [15]:
# Отменённые заказы в файл
today = date.today()
cancelled.to_excel(f'orders_cancelled_{today}.xlsx', index=False)

In [79]:
# Топ товаров по отказам с разбивкой по актикулю
t = cancelled.groupby('supplierArticle', as_index=False).agg({'gNumber' : 'count'})  \
    .rename(columns = {'gNumber': 'quant_cancelled'})
m = df.groupby('supplierArticle', as_index=False).agg({'gNumber' : 'count'}).rename(columns = {'gNumber':'total_order'})
s = m.merge(t, how='left', on='supplierArticle').fillna(0)
s['cancel_prc'] = s.quant_cancelled / s.total_order * 100
s.sort_values('cancel_prc', ascending=False)

,supplierArticle,total_order,quant_cancelled,cancel_prc
9,розовежник,3,2.0,66.666667
10,черный журавли,5,2.0,40.000000
8,Этнический халат,85,25.0,29.411765
6,Шелковая пижама белые листья,7,2.0,28.571429
2,Синий халат,35,7.0,20.000000
7,Шелковая пижама лист1,18,3.0,16.666667
0,Голубалдеж,19,3.0,15.789474
1,Красный,2,0.0,0.000000
3,Тигры,2,0.0,0.000000
4,Цвет ЗАРА,4,0.0,0.000000


In [80]:
# Топ товаров по отказам с разбивкой по размерам
t = cancelled.groupby('techSize', as_index=False).agg({'gNumber' : 'count'})  \
    .rename(columns = {'gNumber': 'quant_cancelled'})
m = df.groupby('techSize', as_index=False).agg({'gNumber' : 'count'}).rename(columns = {'gNumber':'total_order'})
s = m.merge(t, how='left', on='techSize').fillna(0)
s['cancel_prc'] = s.quant_cancelled / s.total_order * 100
s.sort_values('cancel_prc', ascending=False)

,techSize,total_order,quant_cancelled,cancel_prc
3,XS-S,9,3.0,33.333333
0,L-XXXL,20,6.0,30.000000
2,XS-L,137,33.0,24.087591
1,M-L,13,2.0,15.384615
4,XS-М,5,0.0,0.000000


In [81]:
# Разбивка количества заказов по месяцам
t = df.sort_values('date_corr').set_index('date_corr').resample(rule='m').techSize.count()  \
    .reset_index().rename(columns={'techSize':'quant_orders'})
t

,date_corr,quant_orders
0,2023-02-28,21
1,2023-03-31,52
2,2023-04-30,27
3,2023-05-31,35
4,2023-06-30,49


In [51]:
# Разбивка количества заказов по неделям
t = df.sort_values('date_corr').set_index('date_corr').resample(rule='w').techSize.count()  \
    .reset_index().rename(columns={'techSize':'quant_orders'})
t

,date_corr,quant_orders
0,2023-02-26,21
1,2023-03-05,4
2,2023-03-12,13
3,2023-03-19,19
4,2023-03-26,11
5,2023-04-02,10
6,2023-04-09,9
7,2023-04-16,5
8,2023-04-23,4
9,2023-04-30,4


In [82]:
# Процент отмены заказов по месяцам
t = df.sort_values('date_corr').set_index('date_corr').resample(rule='m').techSize.count()  \
    .reset_index().rename(columns={'techSize':'quant_orders_total'})
m = df[df.isCancel==True].sort_values('date_corr').set_index('date_corr').resample(rule='m').techSize.count()  \
    .reset_index().rename(columns={'techSize':'quant_orders_cancalled'})
s = m.merge(t, how='left', on='date_corr')
s['prc_cancelled'] = round(s.quant_orders_cancalled / s.quant_orders_total * 100)
s

,date_corr,quant_orders_cancalled,quant_orders_total,prc_cancelled
0,2023-03-31,16,52,31.0
1,2023-04-30,6,27,22.0
2,2023-05-31,12,35,34.0
3,2023-06-30,10,49,20.0


# Разное

In [47]:
# Выгрузка таблицы с перечнем артикулов
df_tov = pd.Series(df.supplierArticle.unique()).to_frame().rename(columns={0:'supplierArticle'})
df_tov['cost'] = 0
#df_tov
df_tov.to_excel('артикулы_себестоимость.xlsx')

,supplierArticle,cost
0,Этнический халат,0
1,Синий халат,0
2,Шелковая пижама лист1,0
3,Шелковая пижама белые листья,0
4,Цвет ЗАРА,0
5,черный журавли,0
6,Черно-белый,0
7,Красный,0
8,Тигры,0
9,Голубалдеж,0


In [100]:
# Таблица для расчёта конверсии заказ / выкуп
today = date.today()
df_orders = df[['odid', 'supplierArticle', 'techSize', 'date_corr']].copy()
df_orders = df_orders.sort_values('date_corr').rename(columns={'date_corr': 'data_order'})
df_orders.to_excel(f'orders_{today}_for_conversion.xlsx', index=False)